In [7]:
import pandas as pd

In [4]:
fx_list = ['EURCZK', 'AUDUSD', 'EURHUF', 'EURNOK', 'EURPLN',
           'EURSEK', 'EURUSD', 'GBPUSD', 'NZDUSD', 'USDCAD',
           'USDCHF', 'USDCNH', 'USDILS', 'USDJPY', 'USDMXN', 
           'USDSGD', 'USDZAR']

In [10]:
for fx in fx_list:
    try:
        cur_data = pd.read_parquet(f'data/{fx}_data.parquet')
        print(fx, cur_data.shape)
    except Exception as e:
        continue 

EURCZK (1151405, 4)
AUDUSD (1176103, 4)
EURHUF (1152347, 4)
EURNOK (1173941, 4)
EURPLN (1153173, 4)
EURSEK (1154995, 4)
EURUSD (1179255, 4)
GBPUSD (1176068, 4)
NZDUSD (1176046, 4)
USDCAD (1175941, 4)
USDCHF (1176588, 4)
USDCNH (1176026, 4)
USDILS (1139866, 4)
USDJPY (1175716, 4)
USDMXN (1174505, 4)
USDSGD (1177216, 4)


In [ ]:
# df = df.sort_values(by='timestamp')
# df = df[~df.bid.isna()]
# df['Roll Adjusted Bid'] = df['bid'] + df['adjustment']
# df['Roll Adjusted Offer'] = df['offer'] + df['adjustment']


# df['mid_price'] = (df['Roll Adjusted Bid'] + df['Roll Adjusted Offer']) / 2

In [ ]:
# Hurst Exponent

In [ ]:
# hurst_value, _, _ = compute_Hc(df['mid_price'], kind='price', simplified=True)
# hurst_value

In [11]:
# for fx in tqdm(["EURHUF", "EURNOK", "EURSEK"]):
#     fx_sql = f"SELECT * from trading.fx_1min_bar where instrument = '\"{fx}\"'"
#     df_fx = pd.read_sql(fx_sql, con=db_connection)
#     df_fx['mid_price'] = (df_fx['bid'] + df_fx['offer'])/2
#     df_fx = df_fx[~df_fx.mid_price.isna()].sort_values(by='timestamp')
#     hurst_value, _, _ = compute_Hc(df_fx['mid_price'], kind='price', simplified=True)
#     print(fx, hurst_value)

In [ ]:
df.groupby(['contract', 'instrument']).agg({'timestamp':['min', 'max']}).reset_index()\
                                                .sort_values(by=('timestamp', 'min'))